----------

------

# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

## Inicialización

In [ ]:
# 1.1 Inicializacion
import pandas as pd
import numpy as np
from scipy import stats as st
import matplotlib.pyplot as plt



## Cargar datos

In [ ]:
# 1.2 Carga de datos

users = pd.read_csv('/datasets/megaline_users.csv')
calls = pd.read_csv('/datasets/megaline_calls.csv')
messages = pd.read_csv('/datasets/megaline_messages.csv')
internet = pd.read_csv('/datasets/megaline_internet.csv')
plans = pd.read_csv('/datasets/megaline_plans.csv')



## Tarifas

In [ ]:
print('USERS\n', users.shape); users.info(); print(users.head(3), '\n')
print('CALLS\n', calls.shape); calls.info(); print(calls.head(3), '\n')
print('MESSAGES\n', messages.shape); messages.info(); print(messages.head(3), '\n')
print('INTERNET\n', internet.shape); internet.info(); print(internet.head(3), '\n')
print('PLANS\n', plans.shape); plans.info(); print(plans.head(3), '\n')

In [ ]:
# 1.4 muestra de informacion

print('INFO PLANS:')
plans.info()
print('\nHEAD PLANS:')
print(plans.head(10))



Para users las columnas de reg_date y churn_date estan como objet, se sonvertiran a datetime:churn_date tiene muchos faltantes (quedaran como NaT). user_id parece PK. plan enlaza con plans.plans_name.
calls en la columna de call_date es objet, datetime.duration en minutos (float), se aplicara la regla del redondeo por llamada(ceil). se verificaranno negativos y se filtrara para 2018, se agregara columna month.
messages en la columna de message_date es objet, datetime. se ocntara por usuario-mes. se agregara month.
internet en la columna de session_date es objet datetime.mb_used(float) debe ser ≥ 0. se sumaran los MB por usuario-mes y luego redondear mensual a GB: ceil(total_mb/1024). tambien se agregara month.
plans las unidades tienen tipos de datos correctos, solo los limites en MB y cobro extra por GB, segun la cuota mensual en USD, se uniran con plan_name - users plan.

en general la conversion de fecha datetime(reg_date, churn_date, call_date, message_date, session_date) y que NaN se convertira a NaT en chrun date, para el rango temporal se confirmara que todas las fechas caen en 2018, se redondearan llamadas y datos tras sumar MB/mes para las validaciones se revisara que no haya negativos en dutation y mb_used, para referencias todos los user_id en calls/messages/internet existen en users, todos los plan estan en plans. para los duplicados se revisara unicidad razonable de ID en cada tabla, por ultimo la preparacion para la agregacion de columna month(año-mes) en calls/messages/internet.

## Corregir datos

In [ ]:
# 1.5 Correguir datos

users['reg_date'] = pd.to_datetime(users['reg_date'], errors='coerce')
users['churn_date'] = pd.to_datetime(users['churn_date'], errors='coerce')
calls['call_date'] = pd.to_datetime(calls['call_date'], errors='coerce')
messages['message_date'] = pd.to_datetime(messages['message_date'], errors='coerce')
internet['session_date'] = pd.to_datetime(internet['session_date'], errors='coerce')

# mantener solamente registros del año 2018

users = users[users['reg_date'].dt.year == 2018]
calls = calls[calls['call_date'].dt.year == 2018]
messages = messages[messages['message_date'].dt.year == 2018]
internet = internet[internet['session_date'].dt.year == 2018]

# eliminar valores irrreales en medidas de uso

calls = calls[calls['duration'] >= 0]
internet = internet[internet['mb_used'] >= 0]

#reindexar tras filtrados

for df in (users, calls, messages, internet):
  
  df.reset_index(drop=True, inplace=True)




## Enriquecer los datos

In [ ]:
# Mes (1-12) para agregar por usuario-mes

calls['month'] = calls['call_date'].dt.month
messages['month'] = messages['message_date'].dt.month
internet['month'] = internet['session_date'].dt.month

# Minutos redondeados por llamada, segun reglas del plan
calls['minutes_ceil'] = np.ceil(calls['duration']).astype(int)

# Ayuda para facturacion de datos: GB incluidos 

plans['gb_included'] = plans['mb_per_month_included'] / 1024.0

# Region para uso en la hipotesis
users['is_ny_nj'] = users['city'].str.contains('NY-NJ', case=False, na=False)

print(plans.columns.tolist())


## Usuarios/as

In [ ]:
# 1.7 usuarios info general

print('INFO USERS:')
users.info()


In [ ]:
# 1.7 usuarios muestra de datos

print('HEAD USERS:')
print(users.head(10))



Se ve que las fechas reg_date esta en datetime; churn_date correctamente como datetime NaT para faltantes, los tipos user_id int, age int, plan y city como texto, hace falta reforsar un poco mas la columna de city (is ny_nj) para poder considerar bien todas las opciones sin generar algun error en el analisis de datos

### Corregir los datos

In [ ]:
# 1.7.1. Corregir los datos (users)

# Normalizar y validar plan

users['plan'] = users['plan'].str.strip().str.lower()
plans['plan_name'] = plans['plan_name'].str.strip().str.lower()
valid_plans = set(plans['plan_name'])
rows_before = len(users)
users = users[users['plan'].isin(valid_plans)]
removed_invalid_plans = rows_before - len(users)

# Eliminar duplicados por user_id

rows_before = len(users)
users = users.drop_duplicates(subset='user_id', keep='first')
removed_dupes = rows_before - len(users)

# Coherencia de fechas: churn_date >= reg_date 

bad_dates_mask = users['churn_date'].notna() & (users['churn_date'] < users['reg_date'])
fixed_bad_dates = int(bad_dates_mask.sum())
users.loc[bad_dates_mask, 'churn_date'] = pd.NaT

print('Planes inválidos eliminados:', removed_invalid_plans)
print('Duplicados user_id eliminados:', removed_dupes)
print('Fechas incoherentes corregidas (churn<reg):', fixed_bad_dates)




### Enriquecer los datos

In [ ]:
# 1.7.2. Enriquecer los datos (users)

# para región NY–NJ
ny_nj_pattern = r'NY-NJ|New York[-–]Newark[-–]Jersey City'
users['is_ny_nj'] = users['city'].str.contains(ny_nj_pattern, case=False, na=False)

print('NY–NJ marcados:', int(users['is_ny_nj'].sum()))

## Llamadas

In [ ]:
# Llamadas -  informacion general y muestra

print('INFO CALLS:')
calls.info()


In [ ]:
print ('HEAD CALLS:')
print(calls.head(10))



la estructura de datos y los tipos son coherentes con lo que indican, las fechas ya estan en formatos y filtrados para el año 2018, solo se detecta la necesidad de posibles chequeos adicionales por id's unicos en llamadas, distribucion en meses 1 a 12 y llamadas muy largas solo como diagnostico.

### Corregir los datos

In [ ]:
# Eliminar dupllicados por ID's

calls = calls.drop_duplicates(subset='id', keep='first').copy()

# Asegurar regla de minutos facturados: ceil si duration>0, si duration==0 entonces 0

calls['minutes_ceil'] = np.where(calls['duration'] > 0, np.ceil(calls['duration']), 0).astype(int)

# Asegurar 'month' coherente con la fecha

calls['mont'] = calls['call_date'].dt.month

# Mantener solo registros con user_id valido (existente en user)

valid_user_ids = set(users['user_id'])
calls = calls[calls['user_id'].isin(valid_user_ids)].copy()


### Enriquecer los datos

In [ ]:

# Contador de llamadas 

calls['call_count'] = 1

# Identificador de año-mes (YYYY-MM)

calls['year_month'] = calls['call_date'].dt.to_period('M').astype(str)

## Mensajes

In [ ]:

print('INFO MESSAGE:')
messages.info()


In [ ]:

print('INFO MESSAGES:')
print(messages.head(10))


En general se ve que la estructura y tipos de datos ya estan corregidos, no hay valores nulos y las fechas tambien son tipo datetime, se puede verificar si no exiten duplicados en user_id y tambien se puede asegurar que los user_id existen en user, se añadira un contador para la navegacion de usuario-mes, tambien se creara una columna year_month para agrupar con los demas DF.


### Corregir los datos

In [ ]:
# Eliminar duplicados por 'id'

messages = messages.drop_duplicates(subset='id', keep='first').copy()

# mantener solo registros con User_id valido

valid_user_ids = set(users['user_id'])
messages = messages[messages['user_id'].isin(valid_user_ids)].copy()

#asegurar month coherente con la fecha

messages['month'] = messages['message_date'].dt.month

### Enriquecer los datos

In [ ]:
# Enriquecer datos

messages['msg_count'] = 1
messages['year_month'] = messages['message_date'].dt.to_period('M').astype(str)


## Internet

In [ ]:

# print('INFO INTERNET:')
internet.info()


In [ ]:

print('INFO INTERNET')
print(internet.head(10))


la estructura de las filas se ve bien, n ose ven nulos, las fechas tambien ya estan corregidas, para los valores de mb_used se asegura que no haya valores negativos y lo clave para la facturacion es el redondeo mensual de GB, lo que se pudiera mejorar es eliminar los duplicados en id, mantener solo user_id presentes en user, y quizas un double check de no negativos en mb por usuario-mes.

### Corregir los datos

In [ ]:
# Eliminar los duplicados de id

internet = internet.drop_duplicates(subset='id', keep='first').copy()

# Mantener solo registros con user_id valido

valid_user_ids = set(users['user_id'])
internet = internet[internet['user_id'].isin(valid_user_ids)].copy()

# Asegurar no-negativos en mb_used

internet = internet[internet['mb_used'] >= 0].copy()

# Asegurar 'month' coherente con la fecha

internet['month'] = internet['session_date'].dt.month

### Enriquecer los datos

In [ ]:
# Identificador de año-mes (YYY-MM) para agregaciones mensuales

internet['year_month'] = internet['session_date'].dt.to_period('M').astype(str)


## Estudiar las condiciones de las tarifas

In [ ]:
# Condiciones de tarifas

print('CONDICIONES DE TARIFAS:')
print(plans[['plan_name',
             'usd_monthly_pay',
             'minutes_included',
             'messages_included',
             'mb_per_month_included',
             'gb_included',
             'usd_per_minute',
             'usd_per_message',
             'usd_per_gb']])


In [ ]:
# Numero de llamadas hechas por cada usuario al mes

calls_per_month = (calls.groupby(['user_id', 'year_month'], as_index=False).agg(calls_count=('id', 'count')))



In [ ]:
# Minutos usados por cada usuario al mes

minutes_per_month = (calls.groupby(['user_id', 'year_month'], as_index=False).agg(minutes_sum=('minutes_ceil', 'sum')))


In [ ]:
# Numero de mensajes enviados por cada usuario al mes

messages_per_month = (messages.groupby(['user_id', 'year_month'], as_index=False).agg(sms_count=('id', 'count')))


In [ ]:
# Volumen del trafico de internet usado por cada usuario al mes (MB)

internet_per_month = (internet.groupby(['user_id', 'year_month'], as_index=False).agg(mb_sum=('mb_used', 'sum')))

In [ ]:
# Fusionar agregados por user_id + yar_month

usage_monthly = calls_per_month.merge(minutes_per_month, on=['user_id','year_month'], how='outer')
usage_monthly = usage_monthly.merge(messages_per_month, on=['user_id','year_month'], how='outer')
usage_monthly = usage_monthly.merge(internet_per_month, on=['user_id','year_month'], how='outer')

# Rellenar faltantes de conteos/sumas con 0 

usage_monthly[['calls_count','minutes_sum','sms_count','mb_sum']] = (
    usage_monthly[['calls_count','minutes_sum','sms_count','mb_sum']].fillna(0))


In [ ]:
# Añadir la información de la tarifa

usage_monthly = usage_monthly.merge(users[['user_id','plan','is_ny_nj']], on='user_id', how='left'
).merge(plans, left_on='plan', right_on='plan_name', how='left')

In [ ]:
# Calcula el ingreso mensual para cada usuario

# Redondeo mensual de datos a GB

usage_monthly['gb_billed'] = np.ceil(usage_monthly['mb_sum'] / 1024.0).astype(int)

# Excedentes

usage_monthly['min_over'] = np.clip(usage_monthly['minutes_sum'] - usage_monthly['minutes_included'], 0, None)
usage_monthly['sms_over'] = np.clip(usage_monthly['sms_count']  - usage_monthly['messages_included'], 0, None)
usage_monthly['gb_over']  = np.clip(usage_monthly['gb_billed']  - usage_monthly['gb_included'], 0, None)

# Ingreso mensual

usage_monthly['revenue_usd'] = (usage_monthly['usd_monthly_pay']
    + usage_monthly['min_over'] * usage_monthly['usd_per_minute']
    + usage_monthly['sms_over'] * usage_monthly['usd_per_message']
    + usage_monthly['gb_over']  * usage_monthly['usd_per_gb'])


In [ ]:
usage_monthly

## Estudio el comportamiento de usuario

### Llamadas

In [ ]:
# Duracion promedio de llamadas por plan y por mes

avg_minutes = (usage_monthly.groupby(['year_month', 'plan'], as_index=False)['minutes_sum'].mean())
avg_minutes_pivot = avg_minutes.pivot(index='year_month', columns='plan', values='minutes_sum').fillna(0)
ax = avg_minutes_pivot.plot(kind='bar', figsize=(12,5))
ax.set_ylabel('Minutos por usuario (promedio)')
ax.set_title('Duracion promedio de llamadas por plan y por mes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Histograma de minutos mensuales por plan

plt.figure(figsize=(10,5))
for p in ['surf', 'ultimate']:
    subset = usage_monthly.loc[usage_monthly['plan']==p, 'minutes_sum']
    plt.hist(subset, bins=30, alpha=0.6, label=p)
plt.xlabel('Minutos mensuales (suma)')
plt.ylabel('Frecuencia')
plt.title('Distribucion de minutos mensuales por plan')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Media, varianza y desviacion estandar de la duracion mensual de llamadas por plan

calls_stats = (usage_monthly.groupby('plan')['minutes_sum']
               .agg(mean='mean', var='var', std='std', count='count'))
print('Estadisticos de minutos mensuales por plan:\n', calls_stats, '\n')



In [ ]:
# Diagrama de caja de minutos mensuales por plan

usage_monthly.boxplot(column='minutes_sum', by='plan', figsize=(8,5))
plt.title('Distribucion de minutos mensuales por plan')
plt.suptitle('')
plt.xlabel('plan')
plt.ylabel('Minutos mensuales (suma)')
plt.tight_layout()
plt.show()


se verifica que el promedio mensual casi identico, 1.7 min en promedio, la dispersion similar, implicacion de tarifa, ambos promedios estan por debajo de los 500 min incluidos en Surf y muy por debajo de los 3K en Ultimate, casi sin excedentes por minutos en ambos planes.

### Mensajes

In [ ]:
# Comprara numero de mensajes por mes

avg_sms = (usage_monthly.groupby(['year_month', 'plan'], as_index=False)['sms_count']
          .mean())
avg_sms_pivot = avg_sms.pivot(index='year_month', columns='plan', values='sms_count').fillna(0)
ax = avg_sms_pivot.plot(kind='bar', figsize=(12,5))
ax.set_ylabel('SMS por usuario (promedio)')
ax.set_title('SMS promedio por plan y por mes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# Histograma de SMS mensuales por plan

plt.figure(figsize=(10,5))
for p in ['surf', 'ultimate']:
    subset = usage_monthly.loc[usage_monthly['plan']==p, 'sms_count']
    plt.hist(subset, bins=30, alpha=0.6, label=p)
plt.xlabel('SMS mensuales')
plt.ylabel('Frecuencia')
plt.title('Distribucion de SMS mensuales por plan')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Estadisticas de SMS por plan

sms_stats = (usage_monthly.groupby('plan')['sms_count']
            .agg(mean='mean', var='var', std='std', count='count'))
print('Estadisticas de SMS mensuales por plan:\n', sms_stats, '\n')

para los SMS se ve que el pormedio mensual algo mayor en Ultimate, surf 31.16 vs ultimate 37.55, variacion del 6.4, la dispersion tambien es muy similar, para la implicacion de tarifa ambos promedios estan muy por debajo de los limites para el excedente son muy poco probables. 

### Internet

In [ ]:
# Comprara el trafico de internet por mes

avg_mb = (usage_monthly
          .groupby(['year_month','plan'], as_index=False)['mb_sum']
          .mean())
avg_mb_pivot = avg_mb.pivot(index='year_month', columns='plan', values='mb_sum').fillna(0)
ax = avg_mb_pivot.plot(kind='bar', figsize=(12,5))
ax.set_ylabel('MB por usuario (promedio)')
ax.set_title('Tráfico de Internet mensual (MB) por plan y por mes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Histograma de tráfico mensual MB por plan

plt.figure(figsize=(10,5))
for p in ['surf','ultimate']:
    subset = usage_monthly.loc[usage_monthly['plan']==p, 'mb_sum']
    plt.hist(subset, bins=30, alpha=0.6, label=p)
plt.xlabel('MB mensuales (suma)')
plt.ylabel('Frecuencia')
plt.title('Distribución de tráfico mensual (MB) por plan')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Estadísticos de tráfico (MB) por plan

internet_stats = (usage_monthly
                  .groupby('plan')['mb_sum']
                  .agg(mean='mean', var='var', std='std', count='count'))
print('Estadísticos de tráfico mensual (MB) por plan:\n', internet_stats, '\n')

El promedio mensual es mayor en Ultimate, surf cuenta con 16.56 GB Vs ultimate con 17.21 GB, la dispersion tambien en muy similar 8 GB y la implicacion de la tarifa en surf se incluyen 15 los cuales casi simpre se superan en 1.56 GB con costo extra de $10 USD, para Ultimate que incluye 30 el promedio esta muy por debajo por lo que los excedentes son muy raros. en general se cuenta con mayor cantidad de datos de Surf lo cual implica mayor cantidad de usuarios o mas meses activos en surf, en cuanto a minutos y sms no parecen generar excedentes relevantes, pero los datos de surf al promediar mas de los 15 GB, probablemente aporta mas ingresis por excedentes de datos que Ultimate. 

## Ingreso

In [ ]:

# Componentes de ingresos

usage_monthly['rev_base'] = usage_monthly['usd_monthly_pay']
usage_monthly['rev_min']  = usage_monthly['min_over'] * usage_monthly['usd_per_minute']
usage_monthly['rev_sms']  = usage_monthly['sms_over'] * usage_monthly['usd_per_message']
usage_monthly['rev_gb']   = usage_monthly['gb_over']  * usage_monthly['usd_per_gb']

# Histograma de ingreso total por plan

bins = np.histogram_bin_edges(usage_monthly['revenue_usd'], bins=30)
surf = usage_monthly.loc[usage_monthly['plan']=='surf', 'revenue_usd']
ult  = usage_monthly.loc[usage_monthly['plan']=='ultimate', 'revenue_usd']

fig, axes = plt.subplots(1, 2, figsize=(12,4), sharex=True, sharey=True)
axes[0].hist(surf, bins=bins)
axes[0].axvline(surf.mean(), linestyle='--', linewidth=1.8)
axes[0].set_title('Surf')
axes[0].set_xlabel('USD'); axes[0].set_ylabel('Frecuencia')

axes[1].hist(ult, bins=bins)
axes[1].axvline(ult.mean(), linestyle='--', linewidth=1.8)
axes[1].set_title('Ultimate')
axes[1].set_xlabel('USD')

fig.suptitle('Ingreso mensual por plan (distribuciones comparables)', y=1.02)
plt.tight_layout()
plt.show()




In [ ]:
# ingreso promedio por componentes y plan

comp_means = (usage_monthly
              .groupby('plan')[['rev_base','rev_min','rev_sms','rev_gb']]
              .mean())
ax = comp_means.plot(kind='bar', stacked=True, figsize=(10,5))
ax.set_ylabel('USD (promedio por usuario-mes)')
ax.set_title('Ingreso promedio por plan desglosado (base, minutos, SMS, GB)')
plt.xticks(rotation=0); plt.tight_layout(); plt.show()

In [ ]:
# Promedio mensual de ingreso por plan 

avg_rev_month = (usage_monthly
                 .groupby(['year_month','plan'], as_index=False)['revenue_usd']
                 .mean())
pivot_rev = avg_rev_month.pivot(index='year_month', columns='plan', values='revenue_usd').fillna(0)
ax = pivot_rev.plot(kind='bar', figsize=(12,5))
ax.set_ylabel('USD (promedio por usuario-mes)')
ax.set_title('Ingreso mensual promedio por plan')
plt.xticks(rotation=45); plt.tight_layout(); plt.show()

# Estadisticas de ingreso por plan

income_stats = (usage_monthly.groupby('plan')['revenue_usd'].agg(mean='mean', var='var', std='std', count='count'))

print('Estadisticas de ingreso mensual por plan:\n', income_stats)


Se verifica que Ultimate muestra un ingreso promedio por usuario-mes mayor de forma consistente en el tiempo, la base (cuota mensual) explica la mayor parte del ingreso de ultimate, en surf el peso relativo viene de los GB excedentes. minutos y SMS aportan poco a los ingresos, operativamente Ultimate maximiza el ingreso por usuario, surf genera ingreso por excedentes de datos si el uso supera los 15 GB, dicho ingreso puede en algun momento bajar cuando el usuario decida cambiar el plan

## Prueba las hipótesis estadísticas

In [ ]:
# Hipotesis 1: ingreso promedio Surf Vs Ultimate

alpha = 0.05

rev_surf = usage_monthly.loc[usage_monthly['plan']=='surf', 'revenue_usd'].dropna()
rev_ult = usage_monthly.loc[usage_monthly['plan']=='ultimate', 'revenue_usd'].dropna()

res_plans = st.ttest_ind(rev_ult, rev_surf, equal_var=False)  # Welch
print('valor p (planes):', res_plans.pvalue)
if res_plans.pvalue < alpha:
    print('Rechazamos la hipótesis nula (ingresos distintos entre planes)')
else:
    print('No podemos rechazar la hipótesis nula (ingresos iguales entre planes)')


In [ ]:
# Hipotesis 2: ingreso promedio Ny-NJ vs otras regiones (Bilateral)

alpha = 0.05

rev_ny_nj = usage_monthly.loc[usage_monthly['is_ny_nj']==True,  'revenue_usd'].dropna()
rev_otros = usage_monthly.loc[usage_monthly['is_ny_nj']==False, 'revenue_usd'].dropna()

res_region = st.ttest_ind(rev_ny_nj, rev_otros, equal_var=False)  # Welch
print('valor p (NY–NJ vs otras):', res_region.pvalue)
if res_region.pvalue < alpha:
    print('Rechazamos la hipótesis nula (ingresos distintos por región)')
else:
    print('No podemos rechazar la hipótesis nula (ingresos iguales por región)')




## Conclusión general

Durante la preparación, convertimos todas las fechas a tipo datetime, filtramos exclusivamente 2018, eliminamos duplicados, validamos que user_id existiera en users, normalizamos los nombres de plan y corregimos incoherencias. Aplicamos exactamente las reglas de negocio: llamadas redondeadas por llamada al minuto superior y tráfico de Internet agregado por usuario-mes con redondeo mensual de MB a GB antes de facturar. A partir de ahí, construimos los ingresos mensuales como cuota base más excedentes de minutos, SMS y GB.

En la descripción del uso, los minutos y los SMS resultaron muy por debajo de los límites incluidos en ambos planes, por lo que rara vez generan excedentes. En cambio, el tráfico de datos sí marca diferencias: los usuarios de Surf promedian 16.6 GB, por encima de los 15 GB incluidos, lo que implica excedentes frecuentes; en Ultimate el promedio 17.2 GB queda muy por debajo de los 30 GB, por lo que casi no hay cargos adicionales. En consecuencia, los ingresos de Ultimate se explican sobre todo por la cuota base, mientras que en Surf el componente relevante adicional proviene de GB excedentes.

En las pruebas de hipótesis (t de Welch, alpha=0.05), comparamos el ingreso promedio por usuario-mes de Surf y Ultimate y obtuvimos p 3.17×10⁻¹⁵, por lo que rechazamos la hipótesis nula de igualdad: el ingreso promedio difiere y, por los descriptivos y gráficos, Ultimate genera más ingreso por usuario-mes. Al contrastar NY–NJ frente al resto del país, p 0.0335 también llevó a rechazar igualdad: existen diferencias regionales en el ingreso promedio (la dirección exacta se observa comparando las medias reportadas en el cuaderno).

En resumen, Ultimate es el plan más rentable en promedio gracias a su ingreso base estable; Surf aporta ingresos por excedentes de datos cuando el uso supera 15 GB, pero no compensa la ventaja sistemática de la cuota de Ultimate. Estas conclusiones se basan en haber aplicado el recorte temporal a 2018, el redondeo conforme a las reglas y la agregación a nivel usuario-mes; además, la clasificación regional se basó en el texto de city, lo que puede hacer que los resultados dependan de cómo estén escritos los nombres de las ciudades/áreas.